In [3]:
cd C:/Users/PC/Jeehyun/ECE6780/Data/vgg/

C:\Users\PC\Jeehyun\ECE6780\Data\vgg


In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [5]:
b_train = 498
m_train = 1190
b_test = 100
m_test = 139
b_val = 137
m_val = 178

mag_fact = 200
nb_train_samples =  b_train + m_train
nb_validation_samples =  b_val + m_val
batch_size = 1

epochs = 10
img_width, img_height = 224, 224

In [6]:
mag_fact = '200'
model_new_name = 'TL_model_' + mag_fact

top_model_weights_path = model_new_name + '.h5'

model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

top_model = Sequential()

top_model.add(Flatten(input_shape=model.output_shape[1:]))

top_model.add(Dense(256))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.25))
top_model.add(Dense(1))
top_model.add(Activation('sigmoid'))

top_model.load_weights(top_model_weights_path)

Model loaded.


In [7]:
def gen_name(mag_fact):
    train_dir = 'breast_' + mag_fact + '_split/train'
    val_dir = 'breast_' + mag_fact + '_split/val'
    model_name = 'bottleneck_fc_model_' + mag_fact + '.h5'
    return train_dir, val_dir, model_name

train_data_dir, validation_data_dir, model_name = gen_name(mag_fact)

In [8]:
newmodel = Model(input= model.input, output= top_model(model.output))

for layer in newmodel.layers[:15]:
    layer.trainable = False
    
newmodel.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  """Entry point for launching an IPython kernel.


In [9]:
newmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

newmodel.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=1,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 1688 images belonging to 2 classes.
Found 315 images belonging to 2 classes.


C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=1, validation_data=<keras.pre..., steps_per_epoch=1688, validation_steps=315)`


Epoch 1/1
1688/1688 [==============================] - 817s 484ms/step - loss: 0.6427 - accuracy: 0.6410 - val_loss: 0.3194 - val_accuracy: 0.5651


In [24]:
validation_data_dir

'breast_200_split/val'

In [12]:
from keras.models import model_from_json
batch_size = 1
fine_features_train = newmodel.predict_generator(train_generator, nb_train_samples//batch_size)
np.save(open('fine_features_train.npy', 'wb'), fine_features_train)

fine_features_val = newmodel.predict_generator(validation_generator, nb_train_samples//batch_size)
np.save(open('fine_features_val.npy', 'wb'), fine_features_val)

model_new_name = 'tuned_TL_' + mag_fact
model_json = newmodel.to_json()

with open(model_new_name + '.json', "w") as json_file:
    json_file.write(model_json)
    newmodel.save_weights(model_new_name + '.h5')

In [14]:
batch_size = 1
epochs = 30
newmodel.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

#fine_features_train = newmodel.predict_generator(train_generator, nb_train_samples//batch_size)
#np.save(open('fine_features_train.npy', 'wb'), fine_features_train)

#fine_features_val = newmodel.predict_generator(validation_generator, nb_train_samples//batch_size)
#np.save(open('fine_features_val.npy', 'wb'), fine_features_val)

model_new_name = 'tuned_TL_' + mag_fact
model_json = newmodel.to_json()

with open(model_new_name + '.json', "w") as json_file:
    json_file.write(model_json)
    newmodel.save_weights(model_new_name + '.h5')

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=30, validation_data=<keras.pre..., steps_per_epoch=1688, validation_steps=315)`
  


Epoch 1/30
1688/1688 [==============================] - 1373s 813ms/step - loss: 0.6111 - accuracy: 0.6955 - val_loss: 0.2232 - val_accuracy: 0.5651
Epoch 2/30
1688/1688 [==============================] - 1944s 1s/step - loss: 0.5574 - accuracy: 0.7174 - val_loss: 0.7249 - val_accuracy: 0.6635
Epoch 3/30
1688/1688 [==============================] - 1945s 1s/step - loss: 0.5073 - accuracy: 0.7435 - val_loss: 0.0024 - val_accuracy: 0.5651
Epoch 4/30
1688/1688 [==============================] - 1636s 969ms/step - loss: 0.4514 - accuracy: 0.7761 - val_loss: 0.0326 - val_accuracy: 0.8254
Epoch 5/30
1688/1688 [==============================] - 931s 551ms/step - loss: 0.3824 - accuracy: 0.8252 - val_loss: 0.0728 - val_accuracy: 0.7587
Epoch 6/30
1688/1688 [==============================] - 707s 419ms/step - loss: 0.3171 - accuracy: 0.8691 - val_loss: 0.0458 - val_accuracy: 0.8952
Epoch 7/30
1688/1688 [==============================] - 706s 418ms/step - loss: 0.3212 - accuracy: 0.8537 - val_lo

In [15]:
newmodel.summary()
intermediate_from_model = newmodel.get_layer("block5_pool").output
bottleneck_model = Model(newmodel.input, intermediate_from_model)
bottleneck_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


fine_bottleneck_feat_train = bottleneck_model.predict_generator(train_generator, nb_train_samples//batch_size)
np.save(open('fine_bottleneck_feat_train.npy', 'wb'), fine_bottleneck_feat_train)

fine_bottleneck_feat_val = bottleneck_model.predict_generator(validation_generator, nb_train_samples//batch_size)
np.save(open('fine_bottleneck_feat_val.npy', 'wb'), fine_bottleneck_feat_val)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [20]:
np.save(open('fine_bottleneck_feat_train.npy', 'wb'), fine_bottleneck_feat_train)


(1688, 7, 7, 512)

In [28]:
fine_bottleneck_feat_train = bottleneck_model.predict_generator(train_generator, nb_train_samples//batch_size)
np.save('fine_bottleneck_feat_train.npy', fine_bottleneck_feat_train)
fine_bottleneck_feat_val = bottleneck_model.predict_generator(validation_generator, nb_validation_samples//batch_size)
np.save('fine_bottleneck_feat_val.npy', fine_bottleneck_feat_val)

In [29]:
fine_bottleneck_feat_val.shape

(315, 7, 7, 512)

In [ ]:
#from keras.models import model_from_json
#model_new_name = 'finetuning_model_' + mag_fact

#model_json = model.to_json()
#with open(model_new_name + '.json', "w") as json_file:
#    json_file.write(model_json)

#model.save_weights(model_new_name + '.h5')


In [ ]:
batch_size = 10
epochs = 1
newmodel.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

model_new_name = 'tuned_TL_' + mag_fact
model_json = newmodel.to_json()

with open(model_new_name + '.json', "w") as json_file:
    json_file.write(model_json)
    newmodel.save_weights(model_new_name + '.h5')